In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [36]:
poblacion_teorica = pd.read_excel("../data/processed/proyeccion_problemas_de_salud.xlsx")
poblacion_teorica["Diagnostico"] = poblacion_teorica.Diagnostico.str.split().str[0]
DIAGS_RELEVANTES = poblacion_teorica.Diagnostico.unique()
poblacion_teorica = poblacion_teorica.set_index("Diagnostico")

In [79]:
egresos_int = pd.read_csv("../data/processed/egresos_torax_mes_y_dia.csv")
egresos_int = egresos_int[egresos_int["DIAG1"].isin(DIAGS_RELEVANTES)]

In [85]:
egresos_pais = pd.read_csv(
    "../data/processed/ranking_nacional_egresos.csv",
    sep=";",
    encoding="latin-1",
    usecols=[
        "ANO_EGRESO",
        "ESTABLECIMIENTO_SALUD",
        "DIAG1",
        "n_egresos",
        "dias_estada_totales",
        "n_int_q",
        "n_muertos",
    ],
)

egresos_pais = egresos_pais[egresos_pais["DIAG1"].isin(DIAGS_RELEVANTES)]

In [86]:
tabla_dinamica_egresos_pais = pd.pivot_table(
    egresos_pais,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_egresos"],
    aggfunc="sum",
    fill_value=0,
    margins=True,
)

tabla_dinamica_egresos_int = pd.pivot_table(
    egresos_int,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_egresos"],
    aggfunc="sum",
    fill_value=0,
    margins=True,
)

In [87]:
tabla_dinamica_egresos_pais.columns = tabla_dinamica_egresos_pais.columns.droplevel(0)
tabla_dinamica_egresos_int.columns = tabla_dinamica_egresos_int.columns.droplevel(0)

In [90]:
poblacion_teorica.merge(
    tabla_dinamica_egresos_pais,
    how="inner",
    left_index=True,
    right_index=True,
    suffixes=("_teoricos", "_egresos_pais"),
)

,Incidencia (Cada 100.000),Incidencia %,País Incidencia,Sexo Incidencia,Edad Incidencia,Estrato,2017_teoricos,2018_teoricos,2019_teoricos,2020_teoricos,...,2012,2013,2014,2015,2016,2017_egresos_pais,2018_egresos_pais,2019_egresos_pais,2020_egresos_pais,All
Diagnostico,,,,,,,,,,,,,,,,,,,,,
C33X,0.1,0.000001,Internacional,Ambos,Todas,Pais,18.419192,18.751405,19.107216,19.458310,...,16,13,13,32,20,25,18,17,14,287
C33X,0.1,0.000001,Internacional,Ambos,Todas,Region Metropolitana,7.508690,7.702891,7.915199,8.125072,...,16,13,13,32,20,25,18,17,14,287
C33X,0.1,0.000001,Internacional,Ambos,Todas,SSMO,1.349920,1.384881,1.423536,1.461235,...,16,13,13,32,20,25,18,17,14,287
C340,6.9,0.000069,Chile,Ambos,Todas,Pais,1270.924248,1293.846945,1318.397904,1342.623390,...,94,129,138,95,144,163,143,148,103,1450
C340,6.9,0.000069,Chile,Ambos,Todas,Region Metropolitana,518.099610,531.499479,546.148731,560.629968,...,94,129,138,95,144,163,143,148,103,1450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T821,3050.0,0.030500,Internacional,Ambos,Pacientes al año con prótesis,Region Metropolitana,NaN,NaN,NaN,NaN,...,128,123,134,167,157,180,167,222,164,2492
T821,3050.0,0.030500,Internacional,Ambos,Pacientes al año con prótesis,SSMO,NaN,NaN,NaN,NaN,...,128,123,134,167,157,180,167,222,164,2492
Z450,NaN,0.000000,NaN,NaN,NaN,Pais,NaN,NaN,NaN,NaN,...,996,1185,1157,1309,1146,718,649,685,521,16695
